In [0]:
import pyspark
import pandas as pd
import functools
import types
import os
import pyspark.sql.functions as f

from pandas import *
from functools import *
from pyspark import *
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *
from datetime import datetime, timedelta
from delta.tables import *

# Iniciar uma sessão Spark
spark = SparkSession.builder.appName("tabelas").getOrCreate()

In [0]:
# criação de variável para filtrar tudo que mudou nas ultimas 48 horas
data_limite = datetime.now() - timedelta(hours=72)

# acessando a base staging e filtrando os registros conforme tempo de análise
df_pedidos = spark.read.table("hive_metastore.asap_mvp.dados_pedidos")

# filtrando as ultimas 48 horas
df_pedidos = df_pedidos.filter(col("ultima_atualizacao")>=data_limite)

# criando tabela temprorária de atualização
if df_pedidos.count() == 0:
   print("Nenhum Update a ser realizado em dim_pedidos")
else:
    # Fazer o filtro no DataFrame df_pedidos
    filtered_df = df_pedidos

    # criando a tabela temporaria para rodar o upsert
    db_name = "asap_update"
    table_name = "dados_pedidos_update"

    filtered_df.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable(f"{db_name}.{table_name}")

    # iniciando o processo de atualização dos dados
    deltaTableOficial = DeltaTable.forPath(spark, 'dbfs:/user/hive/warehouse/asap_mvp.db/dim_pedidos')
    deltaTableUpdates = DeltaTable.forPath(spark, 'dbfs:/user/hive/warehouse/asap_update.db/dados_pedidos_update')

    dfUpdates = deltaTableUpdates.toDF()
    print("Existem:"+str(dfUpdates.count())+" registros para atualizar")
    deltaTableOficial.alias('oficial') \
    .merge(
    dfUpdates.alias('updates'),
        'oficial.id_pedido = updates.id_pedido'
    ) \
    .whenMatchedUpdate(set =
    {
        "oficial.id_pedido": "updates.id_pedido",
        "oficial.codigoCliente": "updates.codigoCliente",
        "oficial.data_criacao": "updates.data_criacao",
        "oficial.ultima_atualizacao": "updates.ultima_atualizacao",
        "oficial.data_planejada": "updates.data_planejada",
        "oficial.data_previsao_entrega": "updates.data_previsao_entrega",
        "oficial.prazo_maximo_entrega": "updates.prazo_maximo_entrega",
        "oficial.tipo_entrega": "updates.tipo_entrega",
        "oficial.tipo_transferencia": "updates.tipo_transferencia",
        "oficial.tipo_operacao": "updates.tipo_operacao",
        "oficial.status_pedido": "updates.status_pedido",
        "oficial.valor_nota": "updates.valor_nota",
        "oficial.estado_origem": "updates.estado_origem",
        "oficial.destino_estado": "updates.destino_estado",
        "oficial.destino_cidade": "updates.destino_cidade",
        "oficial.destino_cep": "updates.destino_cep"
    }
    ) \
    .whenNotMatchedInsert(values =
    {
        "oficial.id_pedido": "updates.id_pedido",
        "oficial.codigoCliente": "updates.codigoCliente",
        "oficial.data_criacao": "updates.data_criacao",
        "oficial.ultima_atualizacao": "updates.ultima_atualizacao",
        "oficial.data_planejada": "updates.data_planejada",
        "oficial.data_previsao_entrega": "updates.data_previsao_entrega",
        "oficial.prazo_maximo_entrega": "updates.prazo_maximo_entrega",
        "oficial.tipo_entrega": "updates.tipo_entrega",
        "oficial.tipo_transferencia": "updates.tipo_transferencia",
        "oficial.tipo_operacao": "updates.tipo_operacao",
        "oficial.status_pedido": "updates.status_pedido",
        "oficial.valor_nota": "updates.valor_nota",
        "oficial.estado_origem": "updates.estado_origem",
        "oficial.destino_estado": "updates.destino_estado",
        "oficial.destino_cidade": "updates.destino_cidade",
        "oficial.destino_cep": "updates.destino_cep"
    }
    ) \
    .execute()

    print("Tabela dim_pedidos atualizada com sucesso")

Existem:9635 registros para atualizar
Tabela dim_pedidos atualizada com sucesso


In [0]:
# criação de variável para filtrar tudo que mudou nas ultimas 48 horas
data_limite = datetime.now() - timedelta(hours=48)

# acessando a base staging e filtrando os registros conforme tempo de análise
df_pedidos_filhos = spark.read.table("hive_metastore.asap_mvp.dados_pedidos_filhos")

# filtrando as ultimas 48 horas
df_pedidos_filhos = df_pedidos_filhos.filter(col("ultima_atualizacao")>=data_limite)

if df_pedidos_filhos.count() == 0:
   print("Nenhum Update a ser realizado")
else:
    # iniciando o processo de atualização dos dados
    tabela_delta = 'dbfs:/user/hive/warehouse/asap_mvp.db/dim_pedidos_filhos'
    df_tabela = spark.read.format('delta').load(tabela_delta)

    df_diff = df_pedidos_filhos.join(df_tabela, (df_pedidos_filhos.id_pedido_filho == df_tabela.id_pedido_filho)\
            & (df_pedidos_filhos.codigoRoteiros == df_tabela.codigoRoteiros), "left_anti")
    
    if df_diff.count() ==0:
        print("Nada a ser inserido")
    else:
        # Realizar o append na tabela Delta
        df_diff.write.format("delta").mode("append").save(tabela_delta)

In [0]:
# criação de variável para filtrar tudo que mudou nas ultimas 48 horas
data_limite = datetime.now() - timedelta(hours=48)

# acessando a base staging e filtrando os registros conforme tempo de análise
df_roteiros = spark.read.table("hive_metastore.asap_mvp.dados_roteiros")

# filtrando as ultimas 48 horas
df_roteiros = df_roteiros.filter(col("lastUpdated")>=data_limite)

# criando tabela temprorária de atualização
if df_roteiros.count() == 0:
   print("Nenhum Update a ser realizado em dim_pedidos")
else:
    # Fazer o filtro no DataFrame df_pedidos
    filtered_df = df_roteiros

    # criando a tabela temporaria para rodar o upsert
    db_name = "asap_update"
    table_name = "dados_roteiros_update"

    filtered_df.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable(f"{db_name}.{table_name}")

    # iniciando o processo de atualização dos dados
    deltaTableOficial = DeltaTable.forPath(spark, 'dbfs:/user/hive/warehouse/asap_mvp.db/dim_roteiros')
    deltaTableUpdates = DeltaTable.forPath(spark, 'dbfs:/user/hive/warehouse/asap_update.db/dados_roteiros_update')

    dfUpdates = deltaTableUpdates.toDF()
    print("Existem:"+str(dfUpdates.count())+" registros para atualizar")
    deltaTableOficial.alias('oficial') \
    .merge(
    dfUpdates.alias('updates'),
        'oficial.codigoRoteiro = updates.codigoRoteiro'
    ) \
    .whenMatchedUpdate(set =
    {
        "oficial.codigoRoteiro": "updates.codigoRoteiro",
        "oficial.date": "updates.date",
        "oficial.dateCreated": "updates.dateCreated",
        "oficial.lastUpdated": "updates.lastUpdated",
        "oficial.placaVeiculo": "updates.placaVeiculo",
        "oficial.qtdPedidos": "updates.qtdPedidos",
        "oficial.qtdVolumes": "updates.qtdVolumes",
        "oficial.distanciaKm": "updates.distanciaKm",
        "oficial.regiaoEntregaGuid": "updates.regiaoEntregaGuid"
    }
    ) \
    .whenNotMatchedInsert(values =
    {
        "oficial.codigoRoteiro": "updates.codigoRoteiro",
        "oficial.date": "updates.date",
        "oficial.dateCreated": "updates.dateCreated",
        "oficial.lastUpdated": "updates.lastUpdated",
        "oficial.placaVeiculo": "updates.placaVeiculo",
        "oficial.qtdPedidos": "updates.qtdPedidos",
        "oficial.qtdVolumes": "updates.qtdVolumes",
        "oficial.distanciaKm": "updates.distanciaKm",
        "oficial.regiaoEntregaGuid": "updates.regiaoEntregaGuid"
    }
    ) \
    .execute()

    print("Tabela dim_roteiros atualizada com sucesso")

In [0]:
# criação de variável para filtrar tudo que mudou nas ultimas 48 horas
data_limite = datetime.now() - timedelta(hours=48)

# acessando a base staging e filtrando os registros conforme tempo de análise
df_roteiros_filhos = spark.read.table("hive_metastore.asap_mvp.dados_roteiros_filhos")

# filtrando as ultimas 48 horas
df_roteiros_filhos = df_roteiros_filhos.filter(col("lastUpdated")>=data_limite)

if df_roteiros_filhos.count() == 0:
   print("Nenhum Update a ser realizado")
else:
    # iniciando o processo de atualização dos dados
    tabela_delta = 'dbfs:/user/hive/warehouse/asap_mvp.db/dim_roteiros_filhos'
    df_tabela = spark.read.format('delta').load(tabela_delta)

    df_diff = df_roteiros_filhos.join(df_tabela, (df_roteiros_filhos.codigoRoteiro == df_tabela.codigoRoteiro)\
            & (df_roteiros_filhos.pedidGuidList == df_tabela.pedidGuidList), "left_anti")
    
    if df_diff.count() ==0:
        print("Nada a ser inserido na tabela de roteiros filho")
    else:
        # Realizar o append na tabela Delta
        df_diff.write.format("delta").mode("append").save(tabela_delta)

In [0]:
# acessando a base staging e filtrando os registros conforme tempo de análise
df_filial = spark.read.table("hive_metastore.asap_mvp.dados_filial")

# criando tabela temprorária de atualização
if df_filial.count() == 0:
   print("Nenhum Update a ser realizado em dim_pedidos")
else:
    # Fazer o filtro no DataFrame df_pedidos
    filtered_df = df_filial

    # criando a tabela temporaria para rodar o upsert
    db_name = "asap_update"
    table_name = "dados_filial_update"

    filtered_df.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable(f"{db_name}.{table_name}")

    # iniciando o processo de atualização dos dados
    deltaTableOficial = DeltaTable.forPath(spark, 'dbfs:/user/hive/warehouse/asap_mvp.db/dim_filial')
    deltaTableUpdates = DeltaTable.forPath(spark, 'dbfs:/user/hive/warehouse/asap_update.db/dados_filial_update')

    dfUpdates = deltaTableUpdates.toDF()
    print("Existem:"+str(dfUpdates.count())+" registros para atualizar")
    deltaTableOficial.alias('oficial') \
    .merge(
    dfUpdates.alias('updates'),
        'oficial.id_filial = updates.id_filial'
    ) \
    .whenMatchedUpdate(set =
    {
        "oficial.id_filial": "updates.id_filial",
        "oficial.nome": "updates.nome",
        "oficial.codigoFilial": "updates.codigoFilial",
        "oficial.nomeFantasia": "updates.nomeFantasia"
    }
    ) \
    .whenNotMatchedInsert(values =
    {
        "oficial.id_filial": "updates.id_filial",
        "oficial.nome": "updates.nome",
        "oficial.codigoFilial": "updates.codigoFilial",
        "oficial.nomeFantasia": "updates.nomeFantasia"
    }
    ) \
    .execute()

    print("Tabela dim_filial atualizada com sucesso")

In [0]:
# criação de variável para filtrar tudo que mudou nas ultimas 48 horas
data_limite = datetime.now() - timedelta(hours=48)

# acessando a base staging e filtrando os registros conforme tempo de análise
df_regiao = spark.read.table("hive_metastore.asap_mvp.dados_regiao")

# filtrando as ultimas 48 horas
df_regiao = df_regiao.filter(col("lastUpdated")>=data_limite)

# criando tabela temprorária de atualização
if df_regiao.count() == 0:
   print("Nenhum Update a ser realizado em dim_pedidos")
else:
    # Fazer o filtro no DataFrame df_pedidos
    filtered_df = df_regiao

    # criando a tabela temporaria para rodar o upsert
    db_name = "asap_update"
    table_name = "dados_regiao_update"

    filtered_df.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable(f"{db_name}.{table_name}")

    # iniciando o processo de atualização dos dados
    deltaTableOficial = DeltaTable.forPath(spark, 'dbfs:/user/hive/warehouse/asap_mvp.db/dim_regiao')
    deltaTableUpdates = DeltaTable.forPath(spark, 'dbfs:/user/hive/warehouse/asap_update.db/dados_regiao_update')

    dfUpdates = deltaTableUpdates.toDF()
    print("Existem:"+str(dfUpdates.count())+" registros para atualizar")
    deltaTableOficial.alias('oficial') \
    .merge(
    dfUpdates.alias('updates'),
        'oficial.id_regiao = updates.id_regiao'
    ) \
    .whenMatchedUpdate(set =
    {
        "oficial.id_regiao": "updates.id_regiao",
        "oficial.codigoRegiao": "updates.codigoRegiao",
        "oficial.filialId": "updates.filialId",
        "oficial.nome": "updates.nome",
        "oficial.uf": "updates.uf"
    }
    ) \
    .whenNotMatchedInsert(values =
    {
        "oficial.id_regiao": "updates.id_regiao",
        "oficial.codigoRegiao": "updates.codigoRegiao",
        "oficial.filialId": "updates.filialId",
        "oficial.nome": "updates.nome",
        "oficial.uf": "updates.uf"
    }
    ) \
    .execute()

    print("Tabela dim_regiao atualizada com sucesso")